# 1 - Prerequisites

In [1]:
# Ensure you have the dependencies for this notebook
%pip install -r model_management.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 5.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.0.4
    Uninstalling click-8.0.4:
      Successfully uninstalled click-8.0.4
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.16.0
    Uninstalling azure-storage-blob-12.16.0:
      Successfully uninstalled azure-storage-blob-12.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-storage-file-datalake 12.11.0 requires azure-storage-blob<13.0.0,>=12.16.0b1, but you have azure-storage-blob 12.13.0 which is incompatible.
ray 2.0.0 requires click<=8.0.4,>=7.0, but you have click 8.1.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Required imports
import mlflow
from sklearn import linear_model

In [3]:
# Define naming conventions
experiment_name = "model-management-examples"
model_name = "mlflow-sample-model"
artifact_path = "regressor"

# 2 - Creating models from an existing run

In [4]:
mlflow.set_experiment(experiment_name=experiment_name)
with mlflow.start_run(run_name="sample-run"):
    reg = linear_model.LinearRegression()
    reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

    mlflow.sklearn.log_model(reg, artifact_path)

2024/05/21 14:07:57 INFO mlflow.tracking.fluent: Experiment with name 'model-management-examples' does not exist. Creating a new experiment.
2024/05/21 14:08:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp4jr35evo/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
# search for the last run of the experiment:

exp = mlflow.get_experiment_by_name(experiment_name)
last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]
print(last_run.info.run_id)

dd7aefd6-9015-4e5b-a324-65a09b6f947e


In [6]:
# register the model using Mlflow client

mlflow.register_model(f"runs:/{last_run.info.run_id}/{artifact_path}", model_name)

Successfully registered model 'mlflow-sample-model'.
2024/05/21 14:08:10 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: mlflow-sample-model, version 1
Created version '1' of model 'mlflow-sample-model'.


<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='', last_updated_timestamp=1716300490030, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>

# 3 - Creating models from assets

In [7]:
reg = linear_model.LinearRegression()
reg.fit([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

mlflow.sklearn.save_model(reg, artifact_path)

In [8]:
# Check the files in the folder
!ls regressor

MLmodel  conda.yaml  model.pkl	python_env.yaml  requirements.txt


In [9]:
# register the model from the local path
mlflow.register_model(f"file://{artifact_path}", model_name)

Registered model 'mlflow-sample-model' already exists. Creating a new version of this model...
2024/05/21 14:12:14 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: mlflow-sample-model, version 2
Created version '2' of model 'mlflow-sample-model'.


<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>

# 4 - Querying models

In [10]:
client = mlflow.tracking.MlflowClient()

### a) Querying all the models in the registry

In [11]:
for model in client.search_registered_models():
    print(f"{model.name}")

azureml_dd7aefd6-9015-4e5b-a324-65a09b6f947e_output_mlflow_log_model_591141088
mlflow-sample-model


In [12]:
client.search_registered_models(f"name='{model_name}'")

[<RegisteredModel: aliases={}, creation_timestamp=1716300489306, description='', last_updated_timestamp=1716300489306, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>], name='mlflow-sample-model', tags={}>]

### b) Getting model versions from registered models

In [13]:
client.search_model_versions(f"name='{model_name}'")

[<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>,
 <ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='', last_updated_timestamp=1716300490030, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>]

### c) Getting specific versions of the model

In [14]:
# get the last registered model version of a given model
client.get_registered_model(model_name)

<RegisteredModel: aliases={}, creation_timestamp=1716300489306, description='', last_updated_timestamp=1716300489306, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>], name='mlflow-sample-model', tags={}>

In [15]:
# Get specific version of the model
client.get_model_version(model_name, version=2)

<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>

# 5 - Model stages

### a) Queying model stages

In [16]:
client.get_model_version_stages(model_name, version="latest")

['None', 'Staging', 'Production', 'Archived']

In [17]:
# See model version stage by getting the model from the registry
client.get_latest_versions(model_name, stages=["Staging"])

[]

In [18]:
client.get_latest_versions(model_name, stages=["None"])

[<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='None', description='', last_updated_timestamp=1716300734663, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>]

### b) Transitioning models

In [19]:
client.transition_model_version_stage(model_name, version=2, stage="Staging")

<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='Staging', description='', last_updated_timestamp=1716300758186, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>

In [20]:
# See model version stage by getting the model from the registry
client.get_latest_versions(model_name, stages=["Staging"])

[<ModelVersion: aliases=[], creation_timestamp=1716300734663, current_stage='Staging', description='', last_updated_timestamp=1716300758186, name='mlflow-sample-model', run_id='', run_link='', source='azureml://artifacts/LocalUpload/240521T141213-46d910b0/regressor', status='READY', status_message='', tags={}, user_id='', version='2'>]

In [ ]:
client.transition_model_version_stage(model_name, version=3, stage="Staging", archive_existing_versions=True)

### c) Loading models from stages

In [21]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/Staging")

2024/05/21 14:12:44 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.4.1, required: mlflow==2.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [22]:
pip install mlflow==2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 31.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: mlflow
    Found existing installation: mlflow 2.4.1
    Uninstalling mlflow-2.4.1:
      Successfully uninstalled mlflow-2.4.1
Note: you may need to restart the kernel to use updated packages.


In [23]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}/Staging")

# 6 - Editing and Deleting Models

### a) Editing Models

In [24]:
# edit model's description
client.update_model_version(model_name, version=1, description="A sample regressor model")

<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='A sample regressor model', last_updated_timestamp=1716301380970, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>

In [25]:
# set model version tag
client.set_model_version_tag(model_name, version="1", key="type", value="regression")

In [26]:
# Get specific version of the model
client.get_model_version(model_name, version=1)

<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='A sample regressor model', last_updated_timestamp=1716301431677, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={'type': 'regression'}, user_id='', version='1'>

In [27]:
# remove model version_tag
client.delete_model_version_tag(model_name, version="1", key="type")

In [28]:
# Get specific version of the model
client.get_model_version(model_name, version=1)

<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='A sample regressor model', last_updated_timestamp=1716301508569, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>

### b) Deleting Model Version

In [32]:
last_version = client.search_model_versions(filter_string=f"name='{model_name}'")[1].version
print(f"Last model version is {last_version}")

Last model version is 1


In [33]:
# Archive model version
client.transition_model_version_stage(name=model_name, version=1, stage="Archived")

<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='Archived', description='A sample regressor model', last_updated_timestamp=1716301657035, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>

In [34]:
# Retrieve Archived model version
client.transition_model_version_stage(name=model_name, version=1, stage="None")

<ModelVersion: aliases=[], creation_timestamp=1716300490030, current_stage='None', description='A sample regressor model', last_updated_timestamp=1716301698414, name='mlflow-sample-model', run_id='dd7aefd6-9015-4e5b-a324-65a09b6f947e', run_link='', source='azureml://westus.api.azureml.ms/mlflow/v2.0/subscriptions/3b57d2fe-08b1-4fe9-b535-f5c4387b9a66/resourceGroups/mlflow-rg98/providers/Microsoft.MachineLearningServices/workspaces/mlflow-ws98/experiments/2b7be778-1dc3-47f8-9dd7-868dbae47f2d/runs/dd7aefd6-9015-4e5b-a324-65a09b6f947e/artifacts/regressor', status='READY', status_message='', tags={}, user_id='', version='1'>

In [35]:
# Delete model version
client.delete_model_version(model_name, version=last_version)